In [24]:



import pandas as pd
import numpy as np
import struct
import numpy as np

import pcaps
# to play with the lib in real time
%load_ext autoreload
%autoreload 2

import glob 
import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
data = "data_usb_light.pcapng"
for data in glob.glob("*.pcapng"):
    df = pcaps.readPCAP_Healson(data).reset_index()
    pcaps.savePCAP(df,data) 

In [66]:
import json
import base64

with open('config.json') as json_file:
    payloads = json.load(json_file)
for k in data.keys():
    payloads[k] = base64.b64decode(payloads[k][1:-1])

In [142]:
ARRAY = b'\x00'
for k in range(64):
    ARRAY += b'\x00' 


def InitDevice():
    dev = usb.core.find(idVendor=0x04B4, idProduct=0x8613)

    if not dev: print("No Device")

    c = 1
    for config in dev:
        print('config', c)
        print('Interfaces', config.bNumInterfaces)
        # The device was getting "Err 16 busy" on my ubuntu
        for i in range(config.bNumInterfaces):
            if dev.is_kernel_driver_active(i):
                dev.detach_kernel_driver(i)
            print(i)
        c+=1

        print(dev)
    try:
        dev.set_configuration()
    except:
        print("Deja fait")

    conf = FX2Config()
    device = FX2Device()
    device.timeout
    return device
    
def BulkOut(device,payload):
    # 
    return device.bulk_write(0x02, payload,timeout=None)

def BulkIn(device,Length):
    # Receive from Bulk
    return device.bulk_read(0x86,Length)
            
def ControlIn(device):
    return device.control_read(0x40, 184, 0, 0, 32,timeout=None)

def ControlOut(device,request,value,index,length):
    return device.control_write( 0x40, request, value, index,ARRAY[:length],timeout=None )

## Inits

def InitOneA(device,TwoBytes): #178 1 to 179 2
    ControlOut(device,178,1,0,16)
    BulkOut(device,TwoBytes)
    ControlOut(device,179,2,0,16)
    return BulkIn(device,512)

def Read512(device):
    ControlOut(device,179,1,0,16)
    return BulkIn(device,512)

def InitOneB(device,TwoBytes): #178 1 to 179 1
    ControlOut(device,178,1,0,16)
    BulkOut(device,TwoBytes)
    return Read512(device)

def BulkOutTwo(device,Two):
    ControlOut(device,178,1,0,16)
    BulkOut(device,Two)

def BulkOutTwo512(device,Two):
    ControlOut(device,178,1,0,16)
    BulkOut(device,Two)
    return Read512(device)

def BulkOutFour(device,Four):
    ControlOut(device,178,2,0,16)
    BulkOut(device,Four)
    
def BulkOutLarge(device,Large):
    ControlOut(device,178,2,0,16)
    if len(Large)<=4096:
        BulkOut(device,Large)
    else:
        NPackets = len(Large)//4096 
        for k in range(NPackets+1):
            if len(Large[4096*k:]) >= 4096:
                #print(k,len(Large[4096*k:4096*(k+1)]))
                BulkOut(device,Large[4096*k:4096*(k+1)])
            else:
                if len(Large[4096*k:]):
                    #print(k,len(Large[4096*k:]))
                    BulkOut(device,Large[4096*k:])
       
    
def Init1004(device,FourBytes):
    BulkOutTwo(device,b'\x10\x04')
    BulkOutFour(device,FourBytes)
    return 1

## 
    
def InitOne(device):               # Commandes 101 à 135
    print(ControlIn(device))       # 101
    InitOneA(device,b'\x10\x0e')   # 103 to 109
    print(ControlIn(device))       # 111
    InitOneB(device,b'\xff\x06')   # 113 to 119
    InitOneB(device,b'\xff\x07')   # 113 to 119
    InitOneB(device,b'\xff\x05')   # 113 to 119
    
def InitTwo(device):               # 121
   
    for k in [b'\x01\x00\x00\x0f',b' \x00\x03\x0f',b'\x00\x00\x03\x0f'
              b'\xe3\x00\x16\x0f', b'\xff\x00\x14\x0f',b'\xc5\x80B\x0f',b'\x0c\x82F\x0f']:
        Init1004(device,k)
    
    BulkOutTwo(device,b'\x1b\x00') # 193
    BulkOutTwo(device,b'\xff\x1b') # 197
    
    Read512(device)                # 201 à 203

    
def InitThree(device): 
    
    for k in [b'#\xe0',b'$\x01']:   # Processing "# et $"
        BulkOutTwo(device,k)
    for k in [b'\xff#',b'\xff$']:
        BulkOutTwo512(device,k)
        
    for k in [b'%\xdf',b'&\x01']:    # Processing "% et &"
        BulkOutTwo(device,k)
    for k in [b'\xff%',b'\xff&']:
        BulkOutTwo512(device,k)
        

    
    
def InitFour(device):             # Starting with TGC
    
    BulkOutTwo(device,b'\x10\x00')     # Command n255
    BulkOutLarge(device,payloads["259"])      # *Array 259
    
    BulkOutTwo(device,b'\x1a\x06')
    BulkOutTwo512(device,b'\xff\x1a')
    
    ## Starting setup parameters
    BulkOutTwo(device,b'\x10\x05')
    BulkOutLarge(device,payloads["279"])     # *Array 279,281,283,285
    
    BulkOutTwo(device,b'\x10\x06')
    BulkOutLarge(device,payloads["293"])     # *Array 293-299
    
    BulkOutTwo(device,b'\x10\x03')
    BulkOutLarge(device,payloads["307"])   # *Array 307
    
    
    BulkOutTwo(device,b'\x10\x0c')
    BulkOutLarge(device,payloads["315"])     # *Array 315-321
    
    BulkOutTwo(device,b'\x10\n')
    BulkOutLarge(device,payloads["329"])   # *Array 329

    BulkOutTwo(device,b'\x10\t')
    BulkOutLarge(device,payloads["337"])      # 337 to 357

    BulkOutTwo(device,b'\x10\x07')
    BulkOutLarge(device,payloads["365"]) # 365

    BulkOutTwo(device,b'\x10\x08')
    BulkOutLarge(device,payloads["373"]) # 373
    
    BulkOutTwo(device,b' \x04')
    BulkOutTwo512(device,b'\xff ')
    
    BulkOutTwo(device,b'\x10\x08')
    BulkOutLarge(device,payloads["393"]) # t
    
    ## Starting few selected commands
    
    for k in [b'+\xe9',b',\t',b'\x17\x01']:
        BulkOutTwo(device,k)    
    BulkOutTwo512(device,b'\xff\x17')
    
    
    for k in [b"'\x19",b'(\x00']:
        BulkOutTwo(device,k)    
    BulkOutTwo512(device,b"\xff'")
    BulkOutTwo512(device,b"\xff'")
    
    
    for k in [b')|',b'*\x01',]:
        BulkOutTwo(device,k)   
    BulkOutTwo512(device,b'\xff)')
    BulkOutTwo512(device,b'\xff*')    

    
    BulkOutTwo(device,b'\x18\x01')
    BulkOutTwo512(device,b'\xff\x18')
    
    BulkOutTwo(device,b'@\x0b')
    BulkOutTwo512(device,b'\xff@')
    
    # A, 01
    BulkOutTwo(device,b'A\x01')
    BulkOutTwo512(device, b'\xffA')
    
    BulkOutTwo512(device, b'\xff\x05')
    
    BulkOutTwo512(device, b'\xff\x05')
    
def InitFive(device):   
    ControlOut(device,182,0,0,16) ## 182, packet ID 515
    
def InitSix(device):
    # Closing!
    for k in [b'\x12\x01',b'\xfe\xaa',b'\x11\x00']:
        BulkOutTwo(device,k)          
    BulkOutTwo512(device, b'\xff\x11')
    
    ControlOut(device,179,0,0,16)

In [143]:
device = InitDevice()

config 1
Interfaces 1
0
DEVICE ID 04b4:8613 on Bus 002 Address 015 =================
 bLength                :   0x12 (18 bytes)
 bDescriptorType        :    0x1 Device
 bcdUSB                 :  0x200 USB 2.0
 bDeviceClass           :    0x0 Specified at interface
 bDeviceSubClass        :    0x0
 bDeviceProtocol        :    0x0
 bMaxPacketSize0        :   0x40 (64 bytes)
 idVendor               : 0x04b4
 idProduct              : 0x8613
 bcdDevice              :    0x0 Device 0.0
 iManufacturer          :    0x1 HealSon
 iProduct               :    0x2 V2.03
 iSerialNumber          :    0x0 
 bNumConfigurations     :    0x1
  CONFIGURATION 1: 100 mA ==================================
   bLength              :    0x9 (9 bytes)
   bDescriptorType      :    0x2 Configuration
   wTotalLength         :   0x20 (32 bytes)
   bNumInterfaces       :    0x1
   bConfigurationValue  :    0x1
   iConfiguration       :    0x0 
   bmAttributes         :   0x80 Bus Powered
   bMaxPower            :  

In [151]:
InitOne(device)
InitTwo(device)
InitThree(device)
InitFour(device)
InitFive(device)
InitSix(device)

bytearray(b'\xb8\x00\x01\x00HEALSON\x00')
bytearray(b'\xb8\x00\x01\x00HEALSON\x00')
